In [0]:
!pip install pymorphy2

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import numpy as np
from itertools import combinations
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()

In [0]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [0]:
pd.set_option('display.max_colwidth', 1000)

In [0]:
from google.colab import drive 
drive.mount('/gdrive')

In [0]:
PATH_TO_DATA = '/gdrive/My Drive/data'

In [0]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [0]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    

###Baseline

In [0]:
def normalize(text): # без pymorphy
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word not in stops]

    return words

In [0]:
data['content_norm'] = data['content'].apply(normalize)

In [0]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [0]:
tfidf.fit(data['content_norm_str'])

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [0]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [0]:
keywords[:3]

In [0]:
evaluate(data['keywords'], keywords)

Результат сильно хуже того, что был на семинаре. Поэтому попробуем применить pymorphy хотя бы к полученным ключевым словам

In [0]:
def norm_words(words):
  normalized = []
  for wordlist in words:
    normal = [morph.parse(word)[0].normal_form for word in wordlist]
    normalized += [normal] 
  return normalized

In [0]:
norm_keywords = norm_words(keywords)

In [0]:
evaluate(data['keywords'], norm_keywords)

Precision -  0.08
Recall -  0.11
F1 -  0.09
Jaccard -  0.05


### Способ 1. Поменяем параметры tfidf

In [0]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5) 

In [0]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=None,
                min_df=2, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [0]:
%%time
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

CPU times: user 2min 57s, sys: 911 ms, total: 2min 58s
Wall time: 2min 58s


In [0]:
norm_keywords = norm_words(keywords)

In [0]:
evaluate(data['keywords'], norm_keywords)

Precision -  0.09
Recall -  0.12
F1 -  0.1
Jaccard -  0.05


Были опробованы различные параметры mindf, maxdf, ngram_range. Результат в некоторых случаях незначительно улучшался(как в примере выше)

###Способ 2. Положение ключевых слов

Нередко в новостных статьях основная мысль озвучивается в первом предложении. Предположим, что ключевые слова чаще всего встречаются именно там

In [0]:
first_sent=data['content'].apply(lambda x: x.split('.')[0])

In [0]:
first_sent.head(5)

0                                                  Действия России, якобы совершившей кибератаки на избирательную комиссию штата Иллинойс в июле 2016 года, должны быть осуждены США
1                                           Президент Латвии Раймонд Вейонис предлагает предоставлять всем детям русскоязычных неграждан латвийское гражданство сразу после рождения
2         Совет Безопасности ООН единогласно принял резолюцию 2401, которая требует от всех сторон конфликта в Сирии безотлагательно остановить боевые действия на территории страны
3                                   Выиграть главный футбольный матч клубного сезона Европы в одиночку невозможно, даже если ты являешься одним из лучших футбольных вратарей в мире
4    Украинским военным стоит быть весьма бдительными, если они всё-таки решат забрать свою технику из Крыма, — ведь Россия вполне может «заминировать корабли», подлежащие передаче
Name: content, dtype: object

Используем функцию normalize из семинарской тетрадки с удалением стоп-слов и несуществительных

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [0]:
first_sent = first_sent.apply(lambda x: normalize(x)[:10]) 

In [0]:
first_sent.head(5)

0                         [действие, россия, кибератака, комиссия, штат, иллинойс, июль, год, сша]
1               [президент, латвия, раймонд, вейонис, ребёнок, негражданин, гражданство, рождение]
2    [совет, безопасность, оон, резолюция, сторона, конфликт, сирия, действие, территория, страна]
3                                                    [матч, сезон, европа, одиночка, вратарь, мир]
4                                  [военный, техника, крым, россия, корабль, подлежащее, передача]
Name: content, dtype: object

In [0]:
evaluate(data['keywords'], first_sent)

Precision -  0.07
Recall -  0.07
F1 -  0.06
Jaccard -  0.04


Результат ухудшился. Однако, на мой взгляд, всё равно он не так уж и плох, учитывая, что в бейзлайне мы анализируем слова во всем тексте, а здесь - берем их только из первого предложения

### Способ 3. Графы

Начнем с тех параметров, которые были на семинаре

In [0]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    ids = [word2id[word] for word in text]

    m = np.zeros((len(vocab), len(vocab)))

    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        for j, k in combinations(window, 2):
            m[j][k] += 1
            m[k][j] += 1
    
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s

    c = Counter()
    n = np.random.choice(len(vocab))
    for i in range(500): 

        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        c.update([n])
    
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n
    


In [0]:
# по какой-то причине 4 статьи были без текста и из-за них всё ломалось
# здесь я их вычислил
i = 0
for element in data['content_norm']:
  if len(element)==0:
    print(i)
  i+=1


In [0]:
data = data.drop([11538, 12728, 12800, 14535]) # удаление пустых текстов

In [0]:
ind = [i for i in range(len(data['keywords']))]
data = data.set_index(pd.Index(ind)) # меняем индексы, чтобы они были по порядку и evaluate работал

In [74]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 28min 3s, sys: 16.3 s, total: 28min 19s
Wall time: 28min 20s


In [0]:
norm_keywords = norm_words(keywords_rw)

In [0]:
ind = [i for i in range(len(data['keywords']))]

In [0]:
data.set_index(pd.Index(ind))

In [102]:
evaluate(data['keywords'], norm_keywords)

Precision -  0.06
Recall -  0.08
F1 -  0.06
Jaccard -  0.03


Получилось хуже бейзлайна

Попробуем задать другие параметры. Сделаем окно - 2 слова

In [0]:
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 2))

In [0]:
norm_keywords = norm_words(keywords_rw)

In [105]:
evaluate(data['keywords'], norm_keywords)

Precision -  0.02
Recall -  0.03
F1 -  0.02
Jaccard -  0.01


окно - 6 слов

In [106]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 2))

CPU times: user 26min 28s, sys: 12.3 s, total: 26min 41s
Wall time: 26min 43s


In [0]:
norm_keywords = norm_words(keywords_rw)

In [108]:
evaluate(data['keywords'], norm_keywords)

Precision -  0.02
Recall -  0.03
F1 -  0.02
Jaccard -  0.01


Выводы:
1) Изменение параметров tfidf может улучшить результат
2) Положение слов в тексте действительно может играть роль при извлечении ключевых слов, но при этом важно учитывать специфику текста
3) Графы могут показать неплохой результат, важен правильный подбор параметров
4) Бейзлайн был незначительно побит только изменением параметров tfidf
5) К сожалению, не хватило времени и ресурсов на лемматизацию основного текста при помощи pymorphy2. Судя по семинару, это могло бы сильно улучшить результат (но датасет там был поменьше)